# 💬 Chat-Commerce 🛒

## Setup

In [ ]:
# Install packages
%pip install python-dotenv langchain llama-index

In [1]:
from dotenv import load_dotenv

# Load environment values
load_dotenv()

True

In [2]:
import logging
import sys

# Turn off unwanted logs
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Index E-Commerce Data

In [4]:
from langchain import OpenAI

# set number of output tokens
num_outputs = 2000

# OpenAI LLM for indexing
llm_index = OpenAI(temperature=0.5, max_tokens=num_outputs)

In [5]:
from pathlib import Path
from llama_index import download_loader

# Read the data from CSV
PagedCSVReader = download_loader("PagedCSVReader")
loader = PagedCSVReader()
documents = loader.load_data(file=Path('./data/home-and-garden.csv'))

In [6]:
from llama_index import GPTSimpleVectorIndex, LLMPredictor, PromptHelper

# set maximum input size
max_input_size = 4096
# set number of output tokens
num_outputs = 2000
# set maximum chunk overlap
max_chunk_overlap = 20
# set chunk size limit
chunk_size_limit = 600 

# Index the data and store the embeddings
# NOTE: Run this only if you want to re-index
llm_predictor = LLMPredictor(llm=llm_index)
prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)

index_data = GPTSimpleVectorIndex(documents, llm_predictor=llm_predictor)
index_data.save_to_disk("./index/index.json")

INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 1815 tokens
> [build_index_from_documents] Total embedding token usage: 1815 tokens


In [ ]:
from llama_index import GPTSimpleVectorIndex

# Load the embeddings from existing index
index_data = GPTSimpleVectorIndex.load_from_disk('./index/index.json')

## Run Queries

In [12]:
from langchain.llms import OpenAIChat

# Initialize OpenAI LLM for chat
llm_chat = OpenAIChat(temperature=0.5)

In [13]:
from langchain.agents import Tool

# Create a langchain 'Tool'
tools = [
    Tool(
        name = "GPT Index",
        func=lambda q: str(index_data.query(q)),
        description="useful for when you want to answer questions about products available in the store. The store sells furnitures and items for home and garden",
        return_direct=True
    ),
]

In [14]:
from llama_index import GPTListIndex
from langchain.agents import initialize_agent
from llama_index.langchain_helpers.memory_wrapper import GPTIndexChatMemory

# Index to be used for chat history
index_chat_history = GPTListIndex([])

# chat history
memory = GPTIndexChatMemory(
    index=index_chat_history, 
    memory_key="chat_history", 
    query_kwargs={"response_mode": "compact"},
    # return_source returns source nodes instead of querying index
    return_source=True,
    # return_messages returns context in message format
    return_messages=True
)

# create an agent
agent_chain = initialize_agent(tools, llm=llm_chat, agent="conversational-react-description", memory=memory)

INFO:root:> [build_index_from_documents] Total LLM token usage: 0 tokens
> [build_index_from_documents] Total LLM token usage: 0 tokens
INFO:root:> [build_index_from_documents] Total embedding token usage: 0 tokens
> [build_index_from_documents] Total embedding token usage: 0 tokens


In [15]:
agent_chain.run(input="hi, i am bob")

INFO:root:> [query] Total LLM token usage: 0 tokens
> [query] Total LLM token usage: 0 tokens
INFO:root:> [query] Total embedding token usage: 0 tokens
> [query] Total embedding token usage: 0 tokens
INFO:root:> [insert] Total LLM token usage: 0 tokens
> [insert] Total LLM token usage: 0 tokens
INFO:root:> [insert] Total embedding token usage: 0 tokens
> [insert] Total embedding token usage: 0 tokens
INFO:root:> [insert] Total LLM token usage: 0 tokens
> [insert] Total LLM token usage: 0 tokens
INFO:root:> [insert] Total embedding token usage: 0 tokens
> [insert] Total embedding token usage: 0 tokens


'Hello Bob! How can I assist you today?'

In [ ]:
agent_chain.run(input="what are the products available?")

In [ ]:
agent_chain.run(input="I'm looking for some furniture to use in my garden")

In [16]:
agent_chain.run(input="Great. Can you pick me one table?")

INFO:root:> [query] Total LLM token usage: 88 tokens
> [query] Total LLM token usage: 88 tokens
INFO:root:> [query] Total embedding token usage: 0 tokens
> [query] Total embedding token usage: 0 tokens
INFO:root:> [query] Total LLM token usage: 155 tokens
> [query] Total LLM token usage: 155 tokens
INFO:root:> [query] Total embedding token usage: 1 tokens
> [query] Total embedding token usage: 1 tokens
INFO:root:> [insert] Total LLM token usage: 0 tokens
> [insert] Total LLM token usage: 0 tokens
INFO:root:> [insert] Total embedding token usage: 0 tokens
> [insert] Total embedding token usage: 0 tokens
INFO:root:> [insert] Total LLM token usage: 0 tokens
> [insert] Total LLM token usage: 0 tokens
INFO:root:> [insert] Total embedding token usage: 0 tokens
> [insert] Total embedding token usage: 0 tokens


'\nThe context information is referring to a bedside table, which is a type of table typically used for placing items such as lamps, alarm clocks, books, and other items next to a bed.'

In [ ]:
agent_chain.run(input="Yes also show me a picture of it")